# Station Ranks

We have seen how the PageRank algorithm works on the trivial problem of identification of a celebrity from a list of people invited to a party. Its now time to put our PageRank algorithm through a real task. The task of obtaining a ranking of all the railway stations of India, based on the amount of traffic flowing through them.

![](assets/Images/railway_station.jpg)

There are more than 8000 railway stations in India. Over 10000 trains pass through these railawy stations to form a large connected network. Our task requires us to rank all the railway stations of India in the order of how busy they are throughout the year.

As you might have obsereved, given such a large sized network, consisting of numerous trains and railway stations, it might prove difficult to assign a metric of ranking to each railway staton which defines how busy it remains throughout the year.
Luckily for us, the PageRank algorithm is quite capable of solving such complex ranking problems, and can provide us with a pretty good ranking of all the train stations in India.

### Importing datasets

To build our network, we make use of two datasets given to us in the form of csv files, <b>trains.csv</b> and <b>stations.csv</b>. These dataset were made available to us courtesy of <b>[Ayush Dubey](https://github.com/ayushdubey003/)</b>.

In [1]:
import numpy as np
import pandas as pd

stations_dataset = pd.read_csv('assets/Datasets/stations.csv')
trains_dataset = pd.read_csv('assets/Datasets/trains.csv')

Pre-viewing stations datatset

In [2]:
stations_dataset.head()

,Station Names,Station Codes,Latitude,Longitude
0,A N DEV NAGAR,ACND,26.7753° N,82.1575° E
1,ABADA,ABB,22.5488° N,88.2035° E
2,ABHAIPUR,AHA,25.2167° N,86.3206° E
3,ABHANPUR JN,AVP,21.0529° N,81.7441° E
4,ABHAYAPURI ASAM,AYU,26.3589° N,90.6485° E


Pre-viewing trains dataset

In [3]:
trains_dataset.iloc[:, :6].head()

,Train Number,Train Name,Running Days,Available Classes,Type,Zone
0,12723,TELANGANA EXP,Daily,1A 2A 3A SL GN,Super Fast,SCR
1,22416,A P EXP,Daily,1A 2A 3A,Super Fast,NR
2,12724,TELANGANA EXP,Daily,1A 2A 3A SL GN,Super Fast,SCR
3,12707,A P SMPRK KRNTI,MON WED FRI,2A 3A SL GN,Super Fast,SCR
4,54582,DLPC NLDM PASS,Daily,UNRESERVED,Passenger,NR


### Building our railway network

We first need to list all the stations in our railway network.

In [4]:
station_table = stations_dataset.values
train_table = trains_dataset.values

station_mapping = {}
for i in range(np.size(station_table, axis=0)):
    station_mapping[station_table[i, 1]] = {'name': station_table[i, 0]}

Next, we will build our railway network by using all the train routes.

In [5]:
edges = []
for i in range(np.size(train_table, axis=0)):
    station_list = train_table[i, 7][:-1].split('$')
    for i in range(len(station_list)):
        if station_list[i] not in station_mapping:
            continue
        else:
            for j in range(i + 1, len(station_list)):
                if station_list[j] not in station_mapping:
                    continue
                else:
                    edges.append((station_list[i], station_list[j], 1))
print(len(edges))

2424929


### Modeling our graph

In [6]:
import sys
sys.path.append('../Implementation')
from graph import Graph

graph = Graph(list(station_mapping.keys()), edges)

### Getting Rankings

In [7]:
ranks = graph.rank(50, 0.85)
rank_list = sorted([(vertex, ranks[vertex]) for vertex in ranks], key=lambda x: -x[1])
for vertex, rank in rank_list:
    print(station_mapping[vertex]['name'], ":", round(rank, 5))

HOWRAH JN : 0.00166
VIJAYAWADA JN : 0.00144
KALYAN JN : 0.00131
KANPUR CENTRAL : 0.00131
ITARSI JN : 0.00126
LUCKNOW NR : 0.00122
AHMEDABAD JN : 0.00119
VADODARA JN : 0.00119
AMBALA CANT JN : 0.00117
THANE : 0.00114
GHAZIABAD : 0.00113
SURAT : 0.00111
LUDHIANA JN : 0.00109
JHANSI JN : 0.00107
BHOPAL  JN : 0.00104
MORADABAD : 0.00104
MATHURA JN : 0.00102
DELHI : 0.00102
VARANASI JN : 0.00101
DD UPADHYAYA JN : 0.00101
BHUSAVAL JN : 0.001
BARDDHAMAN JN : 0.00098
DADAR : 0.00098
H NIZAMUDDIN : 0.00095
NEW DELHI : 0.00095
ANAND JN : 0.00093
GORAKHPUR JN : 0.00092
ASANSOL JN : 0.00092
VISAKHAPATNAM : 0.00091
NAGPUR : 0.00091
SEALDAH : 0.0009
BORIVALI : 0.00089
PATNA JN : 0.00087
KHURDA ROAD JN : 0.00086
JAIPUR : 0.00084
KATPADI JN : 0.00084
BAREILLY : 0.00084
MANMAD JN : 0.00084
C SHIVAJI MAH T : 0.00084
BINA JN : 0.00083
BHUBANESWAR : 0.00083
KHARAGPUR JN : 0.00083
SALEM JN : 0.00083
PUNE JN : 0.00083
GWALIOR : 0.00082
AGRA CANTT : 0.00081
THRISUR : 0.00079
ERODE JN : 0.00079
BANDEL JN : 0.

HOSAPETE JN : 0.00023
BALANGIR : 0.00023
PHAPHUND : 0.00023
TILHAR : 0.00023
SHEGAON : 0.00023
TAPRI : 0.00023
VINDHYACHAL : 0.00023
KASHI : 0.00023
SIDDHPUR : 0.00023
UNJHA : 0.00023
BARGARH ROAD : 0.00023
ALIPUR DUAR JN : 0.00023
ASHOK NAGAR : 0.00023
DAHISAR : 0.00023
KOVVUR : 0.00023
KALKA : 0.00023
MIRA ROAD : 0.00023
BONDAMUNDA : 0.00023
KANPUR BGE L BK : 0.00023
MAGARWARA : 0.00023
SELU : 0.00022
UMARIA : 0.00022
TIRUNINRAVUR : 0.00022
DALSINGH SARAI : 0.00022
MAVLI JN : 0.00022
KENGERI : 0.00022
UCHANA : 0.00022
CHANGANASERI : 0.00022
PUNARAKH : 0.00022
BERCHHA : 0.00022
NANGLOI : 0.00022
DHAMPUR : 0.00022
DIGHWARA : 0.00022
LAKHERI : 0.00022
BOLARUM : 0.00022
ALUABARI ROAD : 0.00022
ADITYAPUR : 0.00022
SAHARSA JN : 0.00022
PITAMBARPUR : 0.00022
PACHORA JN : 0.00022
SEHORE : 0.00022
CHUNAR : 0.00022
CHATRAPUR : 0.00022
SANGLI : 0.00022
KHARSIA : 0.00022
KOT KAPURA : 0.00022
DAYABASTI : 0.00022
VIVEKNNDAPURI H : 0.00022
JALESWAR : 0.00022
DARYABAD : 0.00022
HIJLI : 0.00022
PARVA

KIM : 0.00014
SHAMLI : 0.00014
JHINJHAK : 0.00014
BARAUT : 0.00014
SERAM : 0.00014
NASIRABAD : 0.00014
KRISHNGR CTY JN : 0.00014
KALAPIPAL : 0.00014
GOILKERA : 0.00014
KARAGOLA ROAD : 0.00014
RENUKUT : 0.00014
NANDALUR : 0.00014
BASAI : 0.00014
KANYAKUMARI : 0.00014
JAISALMER : 0.00014
RENGALI : 0.00014
BAGBAHRA : 0.00014
GOWDAVALLI : 0.00014
KHUDIRAM B PUSA : 0.00014
NADBAI : 0.00014
KHERLI : 0.00014
DAHINA ZAINABAD : 0.00014
DONDAICHA : 0.00014
HATHBANDH : 0.00014
PIPRAIGAON : 0.00014
TIRUVERUMBUR : 0.00014
NAWA CITY : 0.00014
PUNDOOAH : 0.00014
KUMHARI : 0.00014
MAHOBA JN : 0.00014
SECBAD JAMES ST : 0.00014
NEKONDA : 0.00014
PALEJ : 0.00014
KHALISPUR : 0.00014
RAMDEVRA : 0.00014
BASNI : 0.00014
MITAWAL : 0.00014
BARANG : 0.00014
KOELWAR : 0.00014
KULHARIA : 0.00014
NEORA : 0.00014
JANKAMPET JN : 0.00014
JANAI ROAD : 0.00014
AMBALAPPUZHA : 0.00014
HARIPPAD : 0.00014
AKODIA : 0.00014
PARASHSHALA : 0.00014
PANDAVAPURA : 0.00014
CHALAMA : 0.00014
UMDANAGAR : 0.00014
BADLAPUR : 0.00014
K

GUNDLA KAMMA : 0.00011
POTLAPADU : 0.00011
KAPAN P H : 0.00011
KHAIRATIYA BH R : 0.00011
TALAIYUTHU : 0.00011
JAPLA : 0.00011
MUSRA : 0.00011
AILAM : 0.00011
SILIARI : 0.00011
VASCO DA GAMA : 0.00011
KALUNGA : 0.00011
SANTAMAGULUR : 0.00011
MUNUMAKA : 0.00011
LAKSHMIBAI NGR : 0.00011
TARUR : 0.00011
KOTMI SONAR P H : 0.00011
KURSELA : 0.00011
MORINDA : 0.00011
MURARPUR : 0.00011
ARALAGUPPE : 0.00011
VATVA : 0.00011
DEVALGAON AUCHR : 0.00011
GARLA : 0.00011
KOTSHILA : 0.00011
PARSODA : 0.00011
KHAJAULI : 0.00011
PANDAUL : 0.00011
KARANJGAON : 0.00011
BAROG : 0.00011
SALOGRA : 0.00011
SOLAN : 0.00011
KANDAGHAT : 0.00011
TATISILWAI : 0.00011
BORIDAND : 0.00011
DHURWASIN P H : 0.00011
HARRAD : 0.00011
MAUHARI : 0.00011
BAIHATOLA : 0.00011
ASLU : 0.00011
DOKWA : 0.00011
DUDHWAKHARA : 0.00011
HADYAL : 0.00011
DAUDPUR : 0.00011
LANJIGARH ROAD : 0.00011
NIDAMANGALAM : 0.00011
GARH DHRUBESWAR : 0.00011
BAGALIA : 0.00011
CHHARRA : 0.00011
KUSHTAUR : 0.00011
CARMELARAM : 0.00011
HELEM : 0.00011
K

DUDIA : 0.0001
BALAWALI : 0.0001
KALLURU : 0.0001
DHEENA : 0.0001
SAINAGAR SHIRDI : 0.0001
RANCHI ROAD : 0.0001
SHRI AMIRGADH : 0.0001
HASANPUR ROAD : 0.0001
BEOHARI : 0.0001
DOBBSPET : 0.0001
MUDDALINGAHALLI : 0.0001
CHIRMIRI : 9e-05
PINDRA ROAD : 9e-05
PENNADAM : 9e-05
MUKURIA : 9e-05
MAHES KHUNT : 9e-05
SHIVAMOGGA H : 9e-05
SUTLANA : 9e-05
CHARAUD : 9e-05
SIWANI : 9e-05
JHUNPA : 9e-05
SEYDUNGANALLUR : 9e-05
NAZARETH : 9e-05
SRIVAIKUNTAM : 9e-05
ARUMUGANERI : 9e-05
KAYALPATTINAM : 9e-05
KURUMBUR : 9e-05
TIRUCHENDUR : 9e-05
THANA BHAWAN : 9e-05
KAPSETI : 9e-05
MAVINKERE : 9e-05
BELAGULA : 9e-05
HOSA AGRAHARA : 9e-05
SAGARKATTE : 9e-05
JAKHAPURA : 9e-05
LATHIDAD : 9e-05
GARHI MANIKPUR : 9e-05
ADAS ROAD : 9e-05
VADOD : 9e-05
SALAIA : 9e-05
NAWADAH : 9e-05
DANKAUR : 9e-05
AJAIBPUR : 9e-05
BORAKI : 9e-05
MARIPAT : 9e-05
BAITARANI ROAD : 9e-05
CANCONA : 9e-05
HIND : 9e-05
CHANDPARA : 9e-05
MANENDRAGARH : 9e-05
BARSI TOWN : 9e-05
USMANABAD : 9e-05
KARANPUR ATO : 9e-05
SNARAYAN CHAPIA : 9e-0

...

THIPPARTHI : 6e-05
VALIGONDA : 6e-05
DHARAMPUR : 6e-05
GODHA : 6e-05
DHANARI : 6e-05
BHAKRAULI : 6e-05
BARSALI : 6e-05
MALKAPUR ROAD : 6e-05
MIRTHAL : 6e-05
DALPATPUR : 6e-05
GEONG : 5e-05
DHINOJ : 5e-05
PANCHOT : 5e-05
RANUJ : 5e-05
SANKHAI : 5e-05
PATAN : 5e-05
NARASINGAPALLI : 5e-05
AJARAKA : 5e-05
PARISAL : 5e-05
GAURIPUR : 5e-05
BARSUAN : 5e-05
BIRMITRAPUR : 5e-05
CHAMPAJHARAM : 5e-05
CHANDIPOSI : 5e-05
DUMERTA : 5e-05
KUARMUNDA : 5e-05
LATHIKATA : 5e-05
PATASAHI : 5e-05
KOTTUR HALT : 5e-05
VENDRA : 5e-05
RATI KA NAGLA : 5e-05
BARHARAGANI : 5e-05
LAKSHMIGANJ : 5e-05
MATHIA BARGHAT : 5e-05
PADRAUNA : 5e-05
RAMKOLA : 5e-05
RAHA : 5e-05
KULALI : 5e-05
ASIRGARH ROAD : 5e-05
BAGMAR : 5e-05
BARGAON GUJAR : 5e-05
BHAIRONPUR : 5e-05
CHANDNI : 5e-05
CHARKHERA : 5e-05
DHARMKUNDI : 5e-05
DONGARGAON : 5e-05
DULARIA : 5e-05
DUSKHEDA : 5e-05
MANDWA : 5e-05
MASANGAON : 5e-05
PAGDHAL : 5e-05
PALASNER : 5e-05
SAGPHATA : 5e-05
WAGHODA : 5e-05
BHIRINGI : 5e-05
DAGARHKERI : 5e-05
CHHARKHERA KURD : 5e

BHORTEX : 5e-05
CHAVALKHEDE : 5e-05
HOL : 5e-05
PALDHI : 5e-05
RANALA : 5e-05
SHARMA : 5e-05
MURUD : 5e-05
SISAUNA : 5e-05
KHARI JHALU : 5e-05
SHERPUR : 5e-05
KODUMMUNDA : 5e-05
PERASHSHANNUR : 5e-05
NAGARI : 5e-05
DUBAHA : 5e-05
KHANUDIH : 5e-05
VENKATAMPALLE : 5e-05
JARAPADA : 5e-05
JUJUMURA : 5e-05
NATHGANJ : 5e-05
GUA : 5e-05
AMBARI : 5e-05
BIMBARI : 5e-05
DEVTHANA : 5e-05
DHANORA DECCAN : 5e-05
JIRONA : 5e-05
JULGAON DECCAN : 5e-05
KOSAI : 5e-05
MAHIMBA : 5e-05
TALMADUGU : 5e-05
THARBAN : 5e-05
UMRAM : 5e-05
CHURARU TAKARLA : 5e-05
DAULATPUR CHAUK : 5e-05
CHINTPURNI MARG : 5e-05
AZARA : 5e-05
CHAYGAON : 5e-05
DUDHNOI : 5e-05
PARADIP : 5e-05
RAHAMA : 5e-05
JAMADOBU : 5e-05
CHICHAKI : 5e-05
NIMIAGHAT : 5e-05
PACHWAN : 5e-05
BAKANIAN BHAUNR : 5e-05
BAKTAL : 5e-05
BOLAI : 5e-05
CHAKROD : 5e-05
JABRI : 5e-05
KISONI : 5e-05
MOHAMMADKHERA : 5e-05
PHANDA : 5e-05
PIR UMROD : 5e-05
BORAWAR : 5e-05
THATHANA MITHRI : 5e-05
BESROLI : 5e-05
GOVINDI MARWAR : 5e-05
GUDHA : 5e-05
NAYA KHARADIA : 5

BARYA RAM : 3e-05
BHANAUR : 3e-05
JARAUNA : 3e-05
SHUDNIPUR : 3e-05
THARWAI : 3e-05
WARIGAON NEWADA : 3e-05
LYALLPUR KC HLT : 3e-05
KALIKIRI : 3e-05
VELLANKANNI : 3e-05
PANOH : 3e-05
EKDIL : 3e-05
PATA : 3e-05
SAMHON : 3e-05
CHEGUNTA : 3e-05
CHIKSUGAR : 3e-05
LINGIRI : 3e-05
SHAMPURHALLI : 3e-05
THANGUNDI : 3e-05
YADLAPUR : 3e-05
YERMARAS : 3e-05
BINAIKI : 3e-05
AHRAURA ROAD : 3e-05
DORAVART CHTRAM : 3e-05
KALASAMUDRAM : 3e-05
KANSRAO : 3e-05
DANAULI PHLWRIA : 3e-05
JANAKINAGAR : 3e-05
PURNIA COURT : 3e-05
MADHA : 3e-05
GONDWANAVISAPUR : 3e-05
ANKUSPUR : 3e-05
DHONDHA DIH : 3e-05
SAHERI : 3e-05
SAIYEDPUR BHTRI : 3e-05
TAJPUR DEHMA : 3e-05
TARAON : 3e-05
GHAZIPUR GHAT : 3e-05
SHAHBAZ KULI : 3e-05
GHATLA : 3e-05
KHANPUR AHIR : 3e-05
MAJRI NANGAL : 3e-05
KHARIO P.H. : 3e-05
ROMPALLE : 3e-05
SALUR : 3e-05
KUMBALAM : 3e-05
GURUJALA : 3e-05
MACHERLA : 3e-05
RENTACHINTALA : 3e-05
BENODA : 3e-05
MASANI : 3e-05
VRINDAVAN : 3e-05
BARALA : 3e-05
LOHAPUR : 3e-05
TAKIPUR : 3e-05
TIKRI : 3e-05
LALAW

GUDRU HALT : 2e-05
GUJJANGIVALASA : 2e-05
GULVANCHI : 2e-05
GUNADALA : 2e-05
GUNDA ROAD JN : 2e-05
GUNGRANA : 2e-05
GUR MARKET : 2e-05
GURHANWA : 2e-05
GURHI : 2e-05
GURUSAR SUTLANI : 2e-05
HADALA BHAL : 2e-05
HADALA KHARI : 2e-05
HAILAKANDI : 2e-05
HALDITA BIHAR : 2e-05
HALGERI : 2e-05
HALLIGUDI : 2e-05
HAMPAPATNAM : 2e-05
HANSARA : 2e-05
HANSIAWAS : 2e-05
HANUMAPURA : 2e-05
HARANGAJAO : 2e-05
HARANYA KHERI : 2e-05
HATIKHALI : 2e-05
HAUSNAGAR : 2e-05
HAZARIBAGH TOWN : 2e-05
HAZRATHPUR HLT : 2e-05
HILARA : 2e-05
HILLIGROVE : 2e-05
HIRE SHELLIKERI : 2e-05
HIWARKHED : 2e-05
HOWBADH JABLPUR : 2e-05
HUDUKULA : 2e-05
HUSAINIWALA : 2e-05
IDAR : 2e-05
IKRA JN : 2e-05
ILAVELANGAL : 2e-05
INDAPUR : 2e-05
INDARGARH : 2e-05
INDORE JN MG : 2e-05
IPPAGUDA : 2e-05
IRANAGALLU : 2e-05
ISIVI : 2e-05
ITAUNJA : 2e-05
JABBALGUDDA : 2e-05
JABLI : 2e-05
JADAR : 2e-05
JAI SAMAND ROAD : 2e-05
JAIMURTINAGAR : 2e-05
JAITPUR KALAN : 2e-05
JAITPURA : 2e-05
JAKLAIR : 2e-05
JALALGARH : 2e-05
JALILA ROAD : 2e-05
JAM

VADORA CBO : 2e-05
VADVIYALA : 2e-05
VAHLYAL : 2e-05
VAJDI ROAD : 2e-05
VAJIRABAD : 2e-05
VAKAV : 2e-05
VALARAMANIKKAM : 2e-05
VALHA : 2e-05
VALLABHNAGAR : 2e-05
VALLIVERU : 2e-05
VANKAL : 2e-05
VANKIYA : 2e-05
VARETHA : 2e-05
VARKHEDI : 2e-05
VASADVA : 2e-05
VASAN IYAWA : 2e-05
VASO : 2e-05
VASTRAPUR : 2e-05
VAVDI : 2e-05
VEDARANNIYAM : 2e-05
VEGANPUR : 2e-05
VEJPUR : 2e-05
VELACHHA : 2e-05
VELANANDAL : 2e-05
VELAVADAR : 2e-05
VELLODU : 2e-05
VEMAR : 2e-05
VEMULURIPADU : 2e-05
VICTOR : 2e-05
VILANGUDI : 2e-05
VILEGAON : 2e-05
VINA : 2e-05
VINAEKMA HALT : 2e-05
VINCHIYA : 2e-05
VIRAMDAD : 2e-05
VIRAVADA : 2e-05
VIRDEL ROAD : 2e-05
VIRKUDI : 2e-05
VIROL : 2e-05
VIRSAD : 2e-05
VIRUL : 2e-05
VISHRAMPURA : 2e-05
VISHWANATH PURI : 2e-05
VYANKATPURA : 2e-05
VYASA COLONY JN : 2e-05
VYASANAKERI : 2e-05
WADALA : 2e-05
WADALI : 2e-05
WADRENGDISA : 2e-05
WADSINGE : 2e-05
WAGHAI : 2e-05
WAGHANIYA : 2e-05
WAGHORIYA : 2e-05
WAN ROAD : 2e-05
WARUDKHED : 2e-05
WASANAPURA : 2e-05
WASHIMBE : 2e-05
YADVE

As we can see, the PageRank algorithm does a pretty decent job of ranking the railway stations. It ranks <b>Howrah Jn</b> as the busiest railway station of India, closely followed by stations like <b>Kanpur Central</b>, <b>Kalyan Jn</b>, <b>Vijaywada Jn</b>, <b>Itarsi Jn</b>, <b>Ahemdabad Jn</b>, <b>Vadodara Jn</b> and <b>Lucknow NR</b>. These predictions seem to be pretty reasonable, and infact align very nicely with the rankings provided by this [website](http://www.walkthroughindia.com/walkthroughs/trains/top-12-busiest-railway-stations-india/).